<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하지 말아주세요.<br>
본 강의를 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b>
</div>

### 0. train/test 데이터 임포트

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import string
import warnings
import missingno
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('titanic/train.csv')
df_test = pd.read_csv('titanic/test.csv')
df_all = pd.concat((df_train, df_test)).reset_index(drop=True)

### 1. 결측치 보정

In [ ]:
missingno.matrix(df_all)

### 1.1. Age

In [ ]:
df_all['Title'] = df_all.Name.str.extract('([A-Za-z]+)\.')
df_all['Title'] = df_all['Title'].replace(['Miss', 'Mrs','Ms', 'Mlle', 'Lady', 'Mme', 'Countess', 'Dona'], 'Mz')
df_all['Title'] = df_all['Title'].replace(['Don', 'Dr','Rev', 'Col', 'Major', 'Capt', 'Sir', 'Jonkheer'], 'Honor')

# 202502 업데이트: 기존 코드는 최신 라이브러리에서 다음 에러가 납니다.
# 기존 코드: df_all['Age'] = df_all.groupby(['Title', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))
# 해당 문제는 apply 대신에 transform 메서드를 사용하면 해결할 수 있습니다.
df_all['Age'] = df_all.groupby(['Title', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))
df_all['Title'].value_counts()

### 1.2. Embarked

In [ ]:
df_all[df_all['Embarked'].isnull()]

In [ ]:
df_all['Ticket']

In [ ]:
df_all[df_all['Cabin'].str.startswith('B2', na=False)]

In [ ]:
# Filling the missing values in Embarked with S
df_all['Embarked'] = df_all['Embarked'].fillna('S')

### 1.3. Fare

<center><img src='https://qph.fs.quoracdn.net/main-qimg-2eaedf5504e843485d9485a773f65bb7.webp'></center>

In [ ]:
df_all[df_all['Fare'].isnull()]

In [ ]:
median_fare = df_all[
    (df_all['Pclass'] == 3) & (df_all['Parch'] == 0) & 
    (df_all['SibSp'] == 0) & (df_all['Embarked'] == 'S')]['Fare'].median()
df_all['Fare'] = df_all['Fare'].fillna(median_fare)

### 1.4. Cabin

In [ ]:
df_all['Cabin'] = df_all['Cabin'].str[0]
df_all['Cabin'] = df_all['Cabin'].fillna('X')
df_all.loc[(df_all['Cabin'] == 'T'), 'Cabin'] = 'A'

In [ ]:
df_all['Cabin'].value_counts()

In [ ]:
missingno.matrix(df_all)

## **2. Feature Engineering**

### **2.1 Binning Continuous Features**

### Fare

In [ ]:
df_all.loc[ df_all['Fare'] <= 20, 'Fare'] = 0
df_all.loc[(df_all['Fare'] > 20) & (df_all['Fare'] <= 40), 'Fare'] = 1
df_all.loc[(df_all['Fare'] > 40) & (df_all['Fare'] <= 60), 'Fare'] = 2
df_all.loc[(df_all['Fare'] > 60) & (df_all['Fare'] <= 80), 'Fare'] = 3
df_all.loc[(df_all['Fare'] > 80) & (df_all['Fare'] <= 100), 'Fare'] = 4
df_all.loc[ df_all['Fare'] > 100, 'Fare'] = 5

### Age

In [ ]:
# 시각화 값에 딱 맞게 설정하는 것은 학습 데이터(train)에만 잘 맞는 overfitting 현상이 발생할 수 있으므로 적절히 적용
df_all.loc[df_all['Age'] <= 2, 'Age'] = 0
df_all.loc[(df_all['Age'] > 2) & (df_all['Age'] <= 15), 'Age'] = 1
df_all.loc[(df_all['Age'] > 15) & (df_all['Age'] <= 20), 'Age'] = 2
df_all.loc[(df_all['Age'] > 20) & (df_all['Age'] <= 33), 'Age'] = 3
df_all.loc[(df_all['Age'] > 33) & (df_all['Age'] <= 43), 'Age'] = 4
df_all.loc[(df_all['Age'] > 43) & (df_all['Age'] <= 60), 'Age'] = 5
df_all.loc[(df_all['Age'] > 60) & (df_all['Age'] <= 75), 'Age'] = 6
df_all.loc[ df_all['Age'] > 75, 'Age'] = 7

### **2.2 Frequency Encoding**

### Family_Size

In [ ]:
df_all['Family_Size'] = df_all['SibSp'] + df_all['Parch'] + 1

### Family_Size_Group

In [ ]:
df_all.loc[df_all['Family_Size'] <= 1, 'Family_Size_Group'] = 'Alone'
df_all.loc[(df_all['Family_Size'] > 1) & (df_all['Family_Size'] <= 4), 'Family_Size_Group'] = 'Small'
df_all.loc[(df_all['Family_Size'] > 4) & (df_all['Family_Size'] <= 6), 'Family_Size_Group'] = 'Medium'
df_all.loc[ df_all['Family_Size'] > 6, 'Family_Size_Group'] = 'Large'

### Ticket_Frequency

In [ ]:
def refine_ticket(data):
    tickets = list()
    for index in range(len(data)):
        ticket = data.iloc[index]
        ticket = ticket.replace(' ', '').strip()
        for punctuation in string.punctuation:
            ticket = ticket.replace(punctuation, '')
        tickets.append(ticket)
    return tickets

In [ ]:
df_all['Ticket_Refinement'] = refine_ticket(df_all['Ticket'])

In [ ]:
df_all['Ticket_Frequency'] = df_all.groupby('Ticket_Refinement')['Ticket_Refinement'].transform('count')

In [ ]:
df_all[df_all['Ticket_Frequency'] == 5]

### Family_Name
- 이름은 Lefebre(Sur Name),(콤마) Master.(Initial) Henry Forbes(First Name)

In [ ]:
print ("<" + string.punctuation + ">")

In [ ]:
def get_surname(data):
    surnames = list()
    for index in range(len(data)):
        name = data.iloc[index].split(',')[0]
        name = name.replace(' ', '').strip()
        for punctuation in string.punctuation:
            name = name.replace(punctuation, '')
        surnames.append(name)
    return surnames

In [ ]:
df_all['Family_Name'] = get_surname(df_all['Name'])
df_train = df_all.loc[:890]
df_test = df_all.loc[891:]

### Family_Survival_Rate & Ticket_Survival_Rate

In [ ]:
# 202502 업데이트: 최신 pandas 라이브러리에서는 컬럼을 선택할 때 리스트 형식을 사용해야 합니다.
# 따라서 다음과 같이 코드를 수정합니다.
# 기존 코드: df_train.groupby('Family_Name')['Survived', 'Family_Size'].median()
df_train.groupby('Family_Name')[['Survived', 'Family_Size']].median()

In [ ]:
# 202502 업데이트: 최신 pandas 라이브러리에서는 컬럼을 선택할 때 리스트 형식을 사용해야 합니다.
# 따라서 다음과 같이 코드를 수정합니다.
# 기존 코드: df_family_survived_rate = df_train.groupby('Family_Name')['Survived', 'Family_Size'].median()
df_family_survived_rate = df_train.groupby('Family_Name')[['Survived', 'Family_Size']].median()

family_rates = dict()
for family_name in df_family_survived_rate.index:
    if df_family_survived_rate.loc[family_name][1] > 1 and (family_name in df_train['Family_Name'].unique() and family_name in df_test['Family_Name'].unique()):
        family_rates[family_name] = df_family_survived_rate.loc[family_name][0]

family_survival_rate = list()
family_survival_rate_NA = list()
mean_survival_rate = np.mean(df_train['Survived'])

for i in range(len(df_all)):
    if df_all['Family_Name'][i] in family_rates:
        family_survival_rate.append(family_rates[df_all['Family_Name'][i]])
        family_survival_rate_NA.append(1)
    else:
        family_survival_rate.append(mean_survival_rate)
        family_survival_rate_NA.append(0)

df_all['Family_Survival_Rate'] = family_survival_rate
df_all['Family_Survival_Rate_NA'] = family_survival_rate_NA        

In [ ]:
# 2025 02 업데이트: 최신 pandas 라이브러리에서는 컬럼을 선택할 때 리스트 형식을 사용해야 합니다.
# 따라서 다음과 같이 코드를 수정합니다.
# 기존 코드: df_ticket_survived_rate = df_train.groupby('Ticket_Refinement')['Survived', 'Ticket_Frequency'].median()
df_ticket_survived_rate = df_train.groupby('Ticket_Refinement')[['Survived', 'Ticket_Frequency']].median()

ticket_rates = dict()
for ticket_name in df_ticket_survived_rate.index:
    if df_ticket_survived_rate.loc[ticket_name][1] > 1 and (ticket_name in df_train['Ticket_Refinement'].unique() and ticket_name in df_test['Ticket_Refinement'].unique()):    
        ticket_rates[ticket_name] = df_ticket_survived_rate.loc[ticket_name][0]

ticket_survival_rate = list()
ticket_survival_rate_NA = list()
mean_survival_rate = np.mean(df_train['Survived'])

for i in range(len(df_all)):
    if df_all['Ticket_Refinement'][i] in ticket_rates:
        ticket_survival_rate.append(ticket_rates[df_all['Ticket_Refinement'][i]])
        ticket_survival_rate_NA.append(1)
    else:
        ticket_survival_rate.append(mean_survival_rate)
        ticket_survival_rate_NA.append(0)

df_all['Ticket_Survival_Rate'] = ticket_survival_rate
df_all['Ticket_Survival_Rate_NA'] = ticket_survival_rate_NA        

### Survival_Rate

In [ ]:
df_all['Total_Survival_Rate'] = (df_all['Ticket_Survival_Rate'] + df_all['Family_Survival_Rate']) / 2
df_all['Total_Survival_Rate_NA'] = (df_all['Ticket_Survival_Rate_NA'] + df_all['Family_Survival_Rate_NA']) / 2
df_train = df_all.loc[:890]
df_test = df_all.loc[891:]

### OneHot Encoding
- Pclass, Sex, Deck, Embarked, Title (Categorical features are converted to One-hot encoding)
- ordinal features are not converted

In [ ]:
onehot_cols = ['Pclass', 'Sex', 'Cabin', 'Embarked', 'Title', 'Family_Size_Group']
df_train = pd.get_dummies(df_train, columns=onehot_cols)
df_test = pd.get_dummies(df_test, columns=onehot_cols)

## **3. Model**

- 컬럼값의 범위가 다른 경우, 모델에도 영향을 미치므로, 값을 균일한 정도로 변경하기 위해, 스케일링을 적용할 수 있음
- 데이터를 모델링하기 전, 스케일링을 통해, 성능 개선 가능
- StandardScaler: 기본 스케일로, 평균과 표준편차 사용하여 스케일링
  - MinMaxScaler: 최대/최소값이 1/0이 되도록 스케일링

In [ ]:
drop_cols = ['Family_Name', 'Family_Size', 'Survived',
             'Name', 'Parch', 'PassengerId', 'SibSp', 'Ticket',
            'Family_Survival_Rate', 'Ticket_Survival_Rate', 
             'Ticket_Survival_Rate_NA', 'Family_Survival_Rate_NA', 'Ticket_Refinement']

In [ ]:
X_train = StandardScaler().fit_transform(df_train.drop(columns=drop_cols))
y_train = df_train['Survived'].values
X_test = StandardScaler().fit_transform(df_test.drop(columns=drop_cols))

### 모델 파라미터 선택
- 모델 파라미터는 노트북 최다하단부의 GridSearchCV() 를 사용한 Hyper parameter 값 사용

In [ ]:
single_best_model = RandomForestClassifier(
                                           n_estimators=1500,
                                           max_depth=6,
                                           min_samples_split=5,
                                           min_samples_leaf=6,
                                           oob_score=True,
                                           random_state=1,
                                           n_jobs=-1,
                                           verbose=True)

### OOB Score 및 생존확률값 계산

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=5, shuffle=True)
probs = pd.DataFrame(
    np.zeros((len(X_test), 5)), 
    columns=['Fold{}'.format(i) for i in range(5)]
)

oob = 0
for fold, indices in enumerate(skf.split(X_train, y_train)):
    train_index, val_index = indices
    single_best_model.fit(X_train[train_index], y_train[train_index])
    oob += single_best_model.oob_score_
    probs['Fold{}'.format(fold)] = single_best_model.predict_proba(X_test)[:, 1] # 각 fold 에서 해당 사람이 Survived 될 확률

print ("OOB Score:", "{:.3f}".format(oob / 5))

probs['avg'] = probs.mean(axis=1) # 해당 사람이 fold 별 Survived 될 확률의 평균 계산
probs['final'] = 0
probs.loc[probs['avg'] > 0.5, 'final'] = 1 # 해당 사람이 fold 별 Survived 될 확률의 평균이 50% 이상일 경우, 최종 생존으로 예측

### 실제 제출

In [ ]:
submission_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
submission_df['PassengerId'] = df_test['PassengerId']
submission_df['Survived'] = probs['final'].tolist()
submission_df.to_csv('submissions_final.csv', header=True, index=False)
submission_df.head(10)

In [ ]:
!kaggle competitions submit -c titanic -f submissions_final.csv -m "Message"

### 4. 참고: Hyperparameter Tuning with GridSearchCV
- 예상범위의 값을 기반으로, GridSearchCV 적용

In [ ]:
from sklearn.model_selection import GridSearchCV

n_estimators = [1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700]
max_depth = [5, 6, 7, 8, 9, 10]
min_samples_split = [5, 6, 7, 8, 9, 10]
min_samples_leaf = [5, 6, 7, 8, 9, 10]

hyperparams = {
    'n_estimators': n_estimators, 
    'max_depth': max_depth, 
    'min_samples_split': min_samples_split, 
    'min_samples_leaf': min_samples_leaf
}

gd=GridSearchCV(
    estimator = RandomForestClassifier(random_state=1), 
    param_grid = hyperparams, 
    verbose=True, 
    cv=5, 
    scoring="accuracy", 
    n_jobs=-1
)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하지 말아주세요.<br>
본 강의를 잘 정리하면, 데이터 분석과 데이터 과학(머신러닝, 인공지능) 모두 가능합니다!<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩</a> 에서 본 강의 기반 최적화된 로드맵도 확인하실 수 있습니다</b>
</div>